In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation, rc
from IPython.display import HTML, display
from numba import jit
import sympy as sp

import rwth_project.numeric_qevolution as nqev
reload(nqev)

import rwth_project.bloch_plotting as bplt
reload(bplt)

import rwth_project.hamiltonians as hams
reload(hams)

rc('animation', embed_limit=1024, bitrate=1024)

In [ ]:
gate_angle = np.pi

w_0 = 1.0
w_D = 1.0
w_R = 1.0
H_avg = 0.2

t_final = gate_angle / (0.5 * H_avg)
t_1 = 0.5*t_final
H_1_linramp = lambda t: np.piecewise(
    t, 
    [
        t < t_1, 
        (t >= t_1) & (t < 2*t_1),
        t >= 2*t_1
    ], 
    [
        lambda ti: (ti/t_1) * 2.0 * H_avg, 
        lambda ti: ((2*t_1 - ti)/t_1) * 2.0 * H_avg,
        0.0
    ]
)

H_1_cosramp = lambda t: (0.5 - 0.5 * np.cos(np.pi*t/t_1))*2.0*H_avg

H_1_static = lambda t: np.ones_like(t) * H_avg

H_1 = H_1_cosramp

ax_R = np.array([0., 0., 1.])

H_perp = [
    lambda t: np.cos(w_D*t)*H_1(t)*0.5, 
    lambda t: np.sin(w_D*t)*H_1(t)*0.0,
    lambda t: np.ones_like(w_0*t) * w_0 * 0.5
]

H_perp_static = [
    lambda t: np.cos(w_D*t)*H_1_static(t)*0.5, 
    lambda t: np.sin(w_D*t)*H_1_static(t)*0.0,
    lambda t: np.ones_like(w_0*t) * w_0 * 0.5
]

H_circ = [
    lambda t: np.cos(w_D*t)*H_1(t)*0.25, 
    lambda t: np.sin(w_D*t)*H_1(t)*0.25,
    lambda t: np.ones_like(w_0*t) * w_0 * 0.5
]

H_countercirc = [
    lambda t: np.cos(-w_D*t)*H_1(t)*0.25, 
    lambda t: np.sin(-w_D*t)*H_1(t)*0.25,
    lambda t: np.ones_like(w_0*t) * w_0 * 0.5
]

psi_0 = np.array([1., 0.], dtype=np.complex128)
psi_0 = psi_0 / np.linalg.norm(psi_0)

rf = True
%time psi_t_perp, plot_t = nqev.integrate_evolution(H_perp, 1e-4, 0.1, t_final, psi_0, rotating_frame=rf, w_R=w_R, ax_R=ax_R)
%time psi_t_circ, _ = nqev.integrate_evolution(H_circ, 1e-4, 0.1,  t_final, psi_0, rotating_frame=rf, w_R=w_R, ax_R=ax_R)
# %time psi_t_pstc, _ = nqev.integrate_evolution(H_perp_static, 1e-4, 0.1,  t_final, psi_0, rotating_frame=rf, w_R=w_R, ax_R=ax_R)

H_t_perp = nqev.assemble_H_paulis(plot_t, *H_perp)
H_t_circ = nqev.assemble_H_paulis(plot_t, *H_circ)
# H_t_countercirc = nqev.assemble_H_paulis(plot_t, *H_countercirc)
# H_t_pstc = nqev.assemble_H_paulis(plot_t, *H_perp_static)

if rf:
    H_t_perp = nqev.transform_rotating_frame(H_t_perp, plot_t, w_R, ax_R)
    H_t_circ = nqev.transform_rotating_frame(H_t_circ, plot_t, w_R, ax_R)
#     H_t_countercirc = nqev.transform_rotating_frame(H_t_countercirc, plot_t, w_R, ax_R)
#     H_t_pstc = nqev.transform_rotating_frame(H_t_pstc, plot_t, w_R, ax_R)

ani = bplt.animate_state_list(
    [psi_t_perp, psi_t_circ], 
    H_t=[H_t_perp, H_t_circ], 
    H_1=[H_1(plot_t)],
    plot_t=plot_t,
    interval=40, 
    trace_qbstate=True,
    trace_rotation=False,
    scale_H=2.0
)

# ani.save("prelim-results/rf-cosramp-pi-H1_%.2f.mp4" % H_avg)

ani_jshtml = ani.to_jshtml()
plt.close()

display(HTML(ani_jshtml))

In [ ]:
gate_angle = np.pi

w_0 = 1.0
w_D = 1.0
w_R = 0.0
H_avg = 0.4

t_final = gate_angle / (0.5 * H_avg)

psi_0 = np.array([1., 0.], dtype=np.complex128)
psi_0 = psi_0 / np.linalg.norm(psi_0)

H1_str = "H_avg*(1 - cos(t/p1))"
p1 = 0.5 * t_final / np.pi

symH_avg, symt, symp1 = sp.symbols("H_avg t p1")
H1_expr = sp.sympify(H1_str).subs([(symH_avg, H_avg), (symp1, p1)])

H1D_funcs = []
for i in range(hams.max_H1_coeff_order + 1):
    H1D_expr = sp.diff(H1_expr, symt, i)
    func = sp.lambdify((symt), H1D_expr, "numpy")
    H1D_funcs.append(lambda t, func=func: np.ones_like(t) * func(t))

H_orig = [
    lambda t: H1D_funcs[0](t) / 4.0 * (1.0 + np.cos(2*w_D*t)),
    lambda t: H1D_funcs[0](t) / 4.0 * (-1.0 * np.sin(2*w_D*t)),
    lambda t: np.ones_like(t) * 0.0,
]

%time psi_t_orig, plot_t = nqev.integrate_evolution(H_orig, 1e-4, 0.1, t_final, psi_0, rotating_frame=False)

# psi_t_approxes = []
# a0 = 1.0
# for i in range(5):
#     Heff_exprs = hams.generate_Heff_expressions(order=i)
#     Heff_funcs = hams.generate_Heff_functions(Heff_exprs, w_D, a0, 0.0, H1D_funcs)
#     %time psi_t_approx, _ = nqev.integrate_evolution(Heff_funcs, 1e-4, 0.1, t_final, psi_0, rotating_frame=False)
#     psi_t_approxes.append(psi_t_approx)

a0s = np.linspace(0, np.pi/2.0, 4)
psi_t_approxes = []
for a0 in a0s:
    Heff_exprs = hams.generate_Heff_expressions(order=1)
    Heff_funcs = hams.generate_Heff_functions(Heff_exprs, w_D, a0, 0.0, H1D_funcs)
    %time psi_t_approx, _ = nqev.integrate_evolution(Heff_funcs, 1e-4, 0.1, t_final, psi_0, rotating_frame=False)
    psi_t_approxes.append(psi_t_approx)

#%time psi_t_circ, _ = nqev.integrate_evolution(H_circ, 1e-4, 0.1,  t_final, psi_0, rotating_frame=False, w_R=w_R, ax_R=ax_R)
# %time psi_t_pstc, _ = nqev.integrate_evolution(H_perp_static, 1e-4, 0.1,  t_final, psi_0, rotating_frame=rf, w_R=w_R, ax_R=ax_R)

H_t_orig = nqev.assemble_H_paulis(plot_t, *H_orig)
# H_t_countercirc = nqev.assemble_H_paulis(plot_t, *H_countercirc)
# H_t_pstc = nqev.assemble_H_paulis(plot_t, *H_perp_static)

ani = bplt.animate_state_list(
    [psi_t_orig] + psi_t_approxes, 
    H_t=[H_t_orig], 
    H_1=[H1D_funcs[0](plot_t)],
    plot_t=plot_t,
    interval=40, 
    trace_qbstate=True,
    trace_rotation=False,
    vector_qbstate=[True] + [False] * len(psi_t_approxes),
    scale_H=1.0,
    blit=True
)

# ani.save("prelim-results/Heff_M2_pi_cosH1_%.2f.mp4" % H_avg)

%time ani_jshtml = ani.to_jshtml()
plt.close()

display(HTML(ani_jshtml))

In [ ]:
np.abs(psi_t_approxes[0][:,0])